In [1]:
!apt-get install -y mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 34 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 

In [2]:
%%writefile hello_mpi.cpp
#include <mpi.h>
#include <stdio.h>
#include <iostream>
using namespace std;

int main(int argc, char** argv) {
    int size, rank;
    MPI_Init(&argc, &argv);                     // Initialize MPI
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);       // Get rank
    MPI_Comm_size(MPI_COMM_WORLD, &size);       // Get total number of processes

    const int arr_size = 5;
    float full_array[arr_size] = { 1.0, 2.0, 3.0, 4.0, 5.0 };
    float result_array[arr_size];

    if (size != 4) {
        if (rank == 0) {
            cout << "This program must be run with 4 processes.\n";
        }
        MPI_Finalize();
        return 0;
    }

    if (rank == 0) {
        for (int i = 0; i < arr_size; i++) {
            float partial_step = full_array[i] - 4;
            MPI_Send(&partial_step, 1, MPI_FLOAT, 1, 0, MPI_COMM_WORLD);
        }
    }
    else if (rank == 1) {
        for (int i = 0; i < arr_size; i++) {
            float partial_step = 0.0;
            MPI_Recv(&partial_step, 1, MPI_FLOAT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            partial_step = partial_step / 5;
            MPI_Send(&partial_step, 1, MPI_FLOAT, 2, 0, MPI_COMM_WORLD);
        }
    }
    else if (rank == 2) {
        for (int i = 0; i < arr_size; i++) {
            float partial_step = 0.0;
            MPI_Recv(&partial_step, 1, MPI_FLOAT, 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            partial_step = partial_step + 3;
            MPI_Send(&partial_step, 1, MPI_FLOAT, 3, 0, MPI_COMM_WORLD);
        }
    }
    else if (rank == 3) {
        cout << "Final result:\n";
        for (int i = 0; i < arr_size; i++) {
            float partial_step = 0.0;
            MPI_Recv(&partial_step, 1, MPI_FLOAT, 2, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            result_array[i] = partial_step * 7;
            cout << result_array[i] << " ";
        }
        cout << endl;
    }

    MPI_Finalize();
    return 0;
}

Writing hello_mpi.cpp


In [3]:
!mpic++ hello_mpi.cpp -o hello_mpi


In [5]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --oversubscribe -np 4 ./hello_mpi


Final result:
16.8 18.2 19.6 21 22.4 
